<a href="https://colab.research.google.com/github/AhmadHegazy/CoronaVirus_Analysis/blob/Development/CoronaVirus_Web_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from datetime import date
today = date.today()

d = today.strftime("%m-%d-%y")
print("date =", d)

date = 03-30-20


In [0]:
cnn_url="https://www.cnn.com/world/live-news/coronavirus-outbreak-{}-intl-hnk/index.html".format(d)
print(cnn_url)

In [0]:
from bs4 import BeautifulSoup
import requests

In [0]:
html = requests.get(cnn_url).text

In [0]:
soup = BeautifulSoup(html)
print(soup.title)

<title data-rh="true">Coronavirus live updates: Cases top 723,000 globally - CNN</title>


In [0]:
import spacy

In [0]:
nlp = spacy.load('en_core_web_sm')

In [0]:
for link in soup.find_all("h2"):
  print("Headline: {}".format(link.text))
  for entity in nlp(link.text).ents:
    print("\tText: {}, Etity: {}".format(entity.text, entity.label_))

In [0]:
#read h2 from html response, descriiption from xml
dw_url = 'https://www.dw.com/en/top-stories/coronavirus/s-32798'
dailymail_url='https://www.dailymail.co.uk/news/coronavirus/index.html'
cnbc_url = 'https://www.cnbc.com/id/10000108/device/rss/rss.html'

In [0]:
urls = [dw_url, dailymail_url, cnn_url, cnbc_url] #cnbc_url
formats = ['html.parser','html.parser','html.parser', 'xml'] #'xml'
tags = ['h2','h2','h2', 'description']
websites = ['DW', 'DailyMail','CNN', 'CNBS']

In [0]:
crawl_index = 0
for url in urls:
  print("\tCrawling web page............{}".format(url))
  print("\t crawl_index: {}".format(crawl_index))
  response = requests.get(url)
  soup = BeautifulSoup(response.content, formats[crawl_index])
  for link in soup.find_all(tags[crawl_index]):
    if('coronavirus' or 'covid-19') in link.text.lower():
      if(len(link.text.split(" ")) > 4):
        print("Headline : {}".format(link.text))

        for entity in nlp(link.text).ents:
          print("\tText: {}, Etity: {}".format(entity.text, entity.label_))

  crawl_index = crawl_index + 1     

In [0]:
crawl_index = 0
news_dict = []
for url in urls:
  print("\tCrawling web page............{}".format(url))
  response = requests.get(url)
  soup = BeautifulSoup(response.content, formats[crawl_index])
  for link in soup.find_all(tags[crawl_index]):
    if('coronavirus' or 'covid-19') in link.text.lower():
      if(len(link.text.split(" ")) > 4):
        print("Headline : {}".format(link.text))
        entities=[]
        entities=[(entity.text, entity.label_) for entity in nlp(link.text).ents]
        entities
        news_dict.append({'website':websites[crawl_index],'url': url,'headline':link.text,'entities':entities})

  crawl_index = crawl_index + 1   

In [0]:
news_dict

In [0]:
import pandas as pd

In [0]:
news_df = pd.DataFrame(news_dict)

In [0]:
news_df.head(20)

In [0]:
news_df[news_df['website'] == 'DW']

In [0]:
#to flateen entities into colums will use apply(pd.Series)
news_final_df = pd.concat([news_df[['website','url','headline']],news_df['entities'].apply(pd.Series)],axis=1)

In [0]:
pd.set_option('max_colwidth', 800)

In [0]:
from google.colab import drivenews_final_df.head()

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


In [0]:
news_df.to_csv('/content/drive/My Drive/Colab Notebooks/CoronaVirus_web_scrapping_dataset.csv')